In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from sqlalchemy import create_engine
%matplotlib inline


In [2]:
df = pd.read_csv('longtimeselling_ctr.csv')

In [3]:
# 2020-02-23
df = df[df['how_long'] <= 60]
df

,pid,how_long,interest,view_cnt,imp_cnt,ctr,pu_num_days,pu_first_day,pu_last_day,pu_pay_free,...,plus_last_day,plus_pay_free,plus_pay_point,plus_pay_sum,vip_num_days,vip_first_day,vip_last_day,vip_pay_free,vip_pay_point,vip_pay_sum
0,117843883,60,5866,4844,70917,0.068305,24.0,2020-03-02,2020-03-26,7280.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,117871954,60,1135,866,13889,0.062352,7.0,2020-03-25,2020-04-01,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,117878002,60,4836,3819,274011,0.013937,36.0,2020-03-17,2020-04-22,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,117881192,60,1914,1199,19878,0.060318,6.0,2020-02-25,2020-03-02,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,117881192,60,1915,1199,19878,0.060318,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2344,121038362,22,11915,491,44575,0.011015,NaN,NaN,NaN,NaN,...,2020-04-13,0.0,95000.0,95000.0,NaN,NaN,NaN,NaN,NaN,NaN
2345,121058790,22,3226,248,94076,0.002636,NaN,NaN,NaN,NaN,...,2020-04-12,0.0,16000.0,16000.0,NaN,NaN,NaN,NaN,NaN,NaN
2346,121136549,21,1045,62,7234,0.008571,NaN,NaN,NaN,NaN,...,2020-04-22,0.0,16000.0,16000.0,NaN,NaN,NaN,NaN,NaN,NaN
2347,121566941,16,1046,157,6904,0.022740,NaN,NaN,NaN,NaN,...,2020-04-11,0.0,31000.0,31000.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
ids = ','.join('\''+str(i)+'\'' for i in df['pid'].tolist())

In [8]:
q = f'''
select left(category_id, 3) as category, count(*)
from service1_quicket.product_info 
where id in ({ids})
group by left(category_id, 3)
'''

category = pd.read_sql(q,con=redshift)

In [10]:
category.sort_values('category')

,category,count
8,100,1
6,200,22
14,210,12
16,230,1
7,240,3
0,310,294
11,320,452
10,400,534
13,410,40
15,500,5


In [14]:
q = f'''
select
   imp_content_id as content_id,
   date,
   count(*) as imp_cnt,
   coalesce(count(view_content_id), 0) as view_cnt,
   cast(view_cnt as float) / cast(imp_cnt as float) as ctr 
from
   (
      SELECT
         i.content_id AS imp_content_id,
         v.content_id AS view_content_id,
         i.year || i.month || i.day as date 
      FROM
         bun_log_db.app_event_type_impression i 
         LEFT OUTER JOIN
            bun_log_db.app_event_type_view v 
            ON v.imp_id = i.imp_id 
      WHERE
         i.YEAR || i.MONTH || i.DAY >= '20200224' 
         and i.year || i.month || i.day <= '20200423' 
         and i.content_id in ({ids})
   )
group by
   imp_content_id,
   date
'''
daily_ctr = pd.read_sql(q, con=redshift)

In [16]:
daily_ctr.to_csv('dail_ctr.csv', index=False)

In [20]:
daily_ctr['ctr'] = (daily_ctr['ctr']*100).round(2)
daily_ctr.head()

,content_id,date,imp_cnt,view_cnt,ctr
0,119679596,20200402,155,30,19.35
1,120463359,20200402,5452,306,5.61
2,118566791,20200402,608,33,5.43
3,120907105,20200402,1863,131,7.03
4,120288159,20200402,4099,32,0.78


In [22]:
daily_ctr.groupby(['content_id', 'date'])['ctr'].aggregate('mean').unstack().to_csv('daily_ctr_pivot.csv')

In [11]:
q = '''
select search_term as ref_term, count(*) as search_cnt
from bun_log_db.app_event_type_search
where year||month||day = '20200510'
group by search_term
order by count(*) desc
limit 200
'''

top200_search = pd.read_sql(q,con=redshift)

In [12]:
q = '''
select ref_term, count(*) as imp_ad_cnt
from bun_log_db.app_event_type_impression
where year||month||day = '20200510' 
    and ref_source in ('ad_super_up_shop', 'ad_super_up', 'power_up') 
    and ref_term is not null and ref_term != ''
group by ref_term
order by count(*) desc
limit 200
'''
top200_imp_ad = pd.read_sql(q,con=redshift)

In [55]:
imp_ad_ctr = pd.read_csv('imp_ad_top200_ctr.csv')

In [59]:
imp_ad_ctr.sort_values('imp_cnt', ascending=False).to_csv('')

,ref_term,imp_cnt,view_cnt,ctr
10,닌텐도 스위치,133128,4152,0.031188
52,나이키,104219,2351,0.022558
155,원피스,85537,1930,0.022563
162,아이폰,72959,1465,0.020080
149,시계,67375,1488,0.022085
...,...,...,...,...
120,후드티,8673,116,0.013375
37,문상,8634,373,0.043201
172,신발,8620,212,0.024594
145,샌들,8618,118,0.013692


In [13]:
q = '''
select ref_term, count(*) as imp_nonad_cnt
from bun_log_db.app_event_type_impression
where year||month||day = '20200510' 
    and ref_campaign is null
    and ref_term is not null and ref_term != ''
group by ref_term
order by count(*) desc
limit 200
'''
top200_imp_nonad = pd.read_sql(q,con=redshift)

In [14]:
q = '''
select ref_term, count(*) as view_ad_cnt
from bun_log_db.app_event_type_view
where year||month||day = '20200510' 
    and ref_source in ('ad_super_up_shop', 'ad_super_up', 'power_up') 
    and ref_term is not null and ref_term != ''
group by ref_term
order by count(*) desc
limit 200
'''
top200_view_ad = pd.read_sql(q,con=redshift)

In [15]:
q = '''
select ref_term, count(*) as view_nonad_cnt
from bun_log_db.app_event_type_view
where year||month||day = '20200510'
    and ref_campaign is null
    and ref_term is not null and ref_term != ''
group by ref_term
order by count(*) desc
limit 200
'''
top200_view_nonad = pd.read_sql(q,con=redshift)

In [30]:
q = '''
select case
    when t3.ref_term is not null then t3.ref_term
    else vn.ref_term end as ref_term, t3.search_cnt, t3.imp_ad_cnt, t3.imp_nonad_cnt, t3.view_ad_cnt, vn.view_nonad_cnt
from (
select case
    when t2.ref_term is not null then t2.ref_term
    else va.ref_term end as ref_term, t2.search_cnt, t2.imp_ad_cnt, t2.imp_nonad_cnt, va.view_ad_cnt
from (
select case
    when t1.ref_term is not null then t1.ref_term
    else inon.ref_term end as ref_term, t1.search_cnt, t1.imp_ad_cnt, inon.imp_nonad_cnt
from
(select case 
    when s.ref_term is not null then s.ref_term
    else ia.ref_term end as ref_term, s.search_cnt, ia.imp_ad_cnt
from
(select search_term as ref_term, count(*) as search_cnt
from bun_log_db.app_event_type_search
where year||month||day = '20200510'
group by search_term
order by count(*) desc
limit 200) s
full outer join
(select ref_term, count(*) as imp_ad_cnt
from bun_log_db.app_event_type_impression
where year||month||day = '20200510' 
    and ref_source in ('ad_super_up_shop', 'ad_super_up', 'power_up') 
    and ref_term is not null and ref_term != ''
group by ref_term
order by count(*) desc
limit 200) ia on s.ref_term = ia.ref_term) t1
full outer join
(select ref_term, count(*) as imp_nonad_cnt
from bun_log_db.app_event_type_impression
where year||month||day = '20200510' 
    and ref_campaign is null
    and ref_term is not null and ref_term != ''
group by ref_term
order by count(*) desc
limit 200) inon on t1.ref_term = inon.ref_term
) t2
full outer join
(select ref_term, count(*) as view_ad_cnt
from bun_log_db.app_event_type_view
where year||month||day = '20200510' 
    and ref_source in ('ad_super_up_shop', 'ad_super_up', 'power_up') 
    and ref_term is not null and ref_term != ''
group by ref_term
order by count(*) desc
limit 200) va on t2.ref_term = va.ref_term
) t3
full outer join 
(select ref_term, count(*) as view_nonad_cnt
from bun_log_db.app_event_type_view
where year||month||day = '20200510'
    and ref_campaign is null
    and ref_term is not null and ref_term != ''
group by ref_term
order by count(*) desc
limit 200) vn on t3.ref_term = vn.ref_term
'''
tops = pd.read_sql(q, con=redshift)

In [31]:
tops

,ref_term,search_cnt,imp_ad_cnt,imp_nonad_cnt,view_ad_cnt,view_nonad_cnt
0,원피스,451.0,85455.0,336720.0,1957.0,8956.0
1,아이폰,2819.0,72875.0,304878.0,1528.0,20842.0
2,시계,985.0,67301.0,281394.0,1552.0,9512.0
3,마르지엘라 팔찌,NaN,51609.0,221416.0,3198.0,7898.0
4,스톤아일랜드,892.0,51085.0,198340.0,1482.0,8685.0
...,...,...,...,...,...,...
343,보이져125,NaN,NaN,NaN,326.0,NaN
344,nmax,NaN,NaN,NaN,337.0,NaN
345,강아지,NaN,NaN,NaN,330.0,NaN
346,강아지분양,NaN,NaN,NaN,NaN,2240.0


In [32]:
tops.to_csv('tops.csv', index=False)

In [35]:
from sklearn.preprocessing import MinMaxScaler
tops = tops.sort_values(['search_cnt', 'imp_ad_cnt', 'imp_nonad_cnt', 'view_ad_cnt', 'view_nonad_cnt'], ascending=False)
tops.head()


,ref_term,search_cnt,imp_ad_cnt,imp_nonad_cnt,view_ad_cnt,view_nonad_cnt
275,닌텐도 스위치,5926.0,132937.0,638996.0,4369.0,49155.0
1,아이폰,2819.0,72875.0,304878.0,1528.0,20842.0
279,노트북,2230.0,49308.0,217553.0,2227.0,14006.0
8,자전거,2203.0,39431.0,262860.0,928.0,13576.0
92,닌텐도스위치,1805.0,45996.0,205529.0,1283.0,13861.0


In [37]:
scaler = MinMaxScaler()
# top200_scaled = scaler.fit_transform(tops[['search_cnt', 'imp_ad_cnt', 'imp_nonad_cnt', 'view_ad_cnt', 'view_nonad_cnt']])

In [40]:
top200_scaled = tops

In [42]:
top200_scaled[['search_cnt', 'imp_ad_cnt', 'imp_nonad_cnt', 'view_ad_cnt', 'view_nonad_cnt']] = scaler.fit_transform(top200_scaled[['search_cnt', 'imp_ad_cnt', 'imp_nonad_cnt', 'view_ad_cnt', 'view_nonad_cnt']])

In [46]:
top200_scaled.to_csv('top200_scaled.csv', index=False)

In [49]:
wo_top1 = top200.iloc[1:]

In [52]:
wo_top1
scaler = MinMaxScaler()
wo_top1[['search_cnt', 'imp_ad_cnt', 'imp_nonad_cnt', 'view_ad_cnt', 'view_nonad_cnt']] = scaler.fit_transform(wo_top1[['search_cnt', 'imp_ad_cnt', 'imp_nonad_cnt', 'view_ad_cnt', 'view_nonad_cnt']])

/Users/emilyPark/.pyenv/versions/3.7.5/envs/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/emilyPark/.pyenv/versions/3.7.5/envs/bun-analytics/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [54]:
wo_top1.to_csv('wo_nintendo_scaled.csv', index=False)

In [21]:
ref_term = ','.join('\''+str(i)+'\'' for i in top200['ref_term'])

,ref_term,imp_ad_cnt,imp_nonad_cnt,search_cnt,view_ad_cnt,view_nonad_cnt
0,닌텐도 스위치,132937.0,638996.0,5926.0,4369.0,NaN
1,나이키,104116.0,402293.0,925.0,2437.0,11738.0
2,원피스,85455.0,336720.0,451.0,1957.0,8956.0
3,아이폰,72875.0,304878.0,2819.0,1528.0,20842.0
4,시계,67301.0,281394.0,985.0,1552.0,NaN
...,...,...,...,...,...,...
414,구찌 카드지갑,NaN,NaN,NaN,NaN,2284.0
415,티맥스,NaN,NaN,NaN,NaN,2281.0
416,강아지분양,NaN,NaN,NaN,NaN,2240.0
417,씰스티커,NaN,NaN,NaN,NaN,2173.0


In [ ]:
q = f'''
select
   imp_content_id as content_id,
   date,
   count(*) as imp_cnt,
   coalesce(count(view_content_id), 0) as view_cnt,
   cast(view_cnt as float) / cast(imp_cnt as float) as ctr 
from
   (
      SELECT
         i.content_id AS imp_content_id,
         v.content_id AS view_content_id,
         i.year || i.month || i.day as date 
      FROM
         bun_log_db.app_event_type_impression i 
         LEFT OUTER JOIN
            bun_log_db.app_event_type_view v 
            ON v.imp_id = i.imp_id 
      WHERE
         i.YEAR || i.MONTH || i.DAY >= '20200224' 
         and i.year || i.month || i.day <= '20200423' 
         and i.content_id in ({ids})
   )
group by
   imp_content_id
'''
daily_ctr = pd.read_sql(q, con=redshift)

In [6]:
# top200 = pd.merge(top200_search, top200_imp_ad, on='ref_term', how='outer', suffixes=['_search', '_imp_ad'])
# top200 = pd.merge(top200, top200_imp_nonad, on='ref_term', how='outer')

In [28]:
top200.columns = ['keyword', '검색', '노출_광고', '노출_비광고']

In [8]:
top200.to_csv('top200.csv', index=False)

In [37]:
top200_search['ref_term'] = top200_search['ref_term'].astype(str)
top200_imp_ad['ref_term'] = top200_imp_ad['ref_term'].astype(str)
top200_imp_nonad['ref_term'] = top200_imp_nonad['ref_term'].astype(str)

In [10]:
q = '''
select *
from 
(
select search_term as ref_term, count(*)
from bun_log_db.app_event_type_search
where year||month||day >= '20200401' and year||month||day <= '20200414'
group by search_term
order by count(*) desc
limit 200
) s
OUTER JOIN
(
select ref_term, count(*)
from bun_log_db.app_event_type_impression
where year||month||day >= '20200401' and year||month||day <='20200414' 
    and ref_source in ('ad_super_up_shop', 'ad_super_up', 'power_up') 
    and ref_term is not null and ref_term != ''
group by ref_term
order by count(*) desc
limit 200
) ia on s.ref_term = ia.ref_term
OUTER JOIN
(
select ref_term, count(*)
from bun_log_db.app_event_type_impression
where year||month||day >= '20200401' and year||month||day <='20200414' 
    and ref_campaign is null
    and ref_term is not null and ref_term != ''
group by ref_term
order by count(*) desc
limit 200
) in on s.ref_term = in.ref_term
'''